|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 2:</h2>|<h1>Large language models<h1>|
|<h2>Section:</h2>|<h1>Fine-tune pretrained models<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge HELPER: Impact of freezing neo on fine-tuning<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM

import time
import requests

# vector plots
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [ ]:
# Eleuther's tokenizer
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125m')
tokenizer.pad_token_id = tokenizer.encode(' ')[0]

# -> GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# load in two GPTneo's and push to GPU
modelFreeze =
modelTrain  =

# Exercise 1: Find the most frequent 100 tokens

In [ ]:
# Moby Dick
text = requests.get('https://www.gutenberg.org/cache/epub/2701/pg2701.txt

# summary
print

In [ ]:
# find the most common tokens and print the top-100


In [ ]:
numreps =  10 # number of random repetitions
numtoks = 100 # output length

# initialize
tokenUsage = np.zeros() # [ pre/post , Freeze/Train ]

# random starting tokens
randstarts = torch.


# FREEZE: generate and store tokens
outFreeze = modelFreeze.generate
genTokensFreeze =


# TRAIN: same as above :)

genTokensTrain =


# calculate the percentage
tokenUsage[0,0] =
tokenUsage[0,1] =

In [ ]:
tokenUsage

# Exercise 2: Targeted training

In [ ]:
for name,param in modelFreeze.named_parameters():
  splitstr = name.split('.')
  print(splitstr)

In [ ]:
# TEST: identify QVK weights in layers >5
for name,param in modelFreeze.named_parameters():
  splitstr = name.split('.')
  if (len(splitstr)>5) and (splitstr[3]=='attn'):
    if (int(splitstr[2])>5) and (splitstr[5][0] in 'qvk'):
      print(name)

In [ ]:
for name,param in modelFreeze.named_parameters():

  # split the name by .


  # see if this fits our filter
  if
      print(f'+++ Layer {name} is trainable (.requires_grad = {param.requires_grad}).')

  # otherwise, freeze the layer
  else:

    print(f'--- Layer {name} is frozen (.requires_grad = {param.requires_grad}).')

# Exercise 3: Fine-tune the models

In [ ]:
# FREEZE optimizer
optimizerFreeze = (modelFreeze.parameters(), lr=.0005)

# TRAIN optimizer
optimizerTrain = torch.optim.

In [ ]:
# training parameters
seq_len     = 256 # max sequence length
batch_size  =  16
num_samples = 474

In [ ]:
# initialize losses
losses = np.zeros((num_samples,2))
delta_norm_em = np.zeros((num_samples,2))

# and computation times
timeTrain = 0
timeFreeze = 0


# grab the initial MLP weights for comparison
prev_emFreeze =
prev_emTrain = modelTrain.transformer.h



# and run the training!
for sampli in range(num_samples):

  # get a batch of data
  ix = torch.randint(len(tokens)-seq_len,size=(batch_size,))
  X  = tokens[ix[:,None] + torch.arange(seq_len)].to(device)


  ### --- FREEZE fine-tuning
  # forward pass and get loss
  start_time = time.time() # start the timer
  timeFreeze += -start_time # end the timer and add
  ### ---------------------


  ### --- TRAIN fine-tuning
  # forward pass and get loss
  ### ---------------------



  ### --- matrix norm to assess change in MLP layer
  delta_norm_em[sampli,0] = torch.norm( - )
  prev_emFreeze = # the current matrix becomes the "previous" for the next iteration

  delta_norm_em[sampli,1] = # same for the train model
  prev_emTrain =




  # update progress display
  if sampli%37==0:
    print(f'Sample {sampli:4}/{num_samples}, losses (Freeze/Train): {losses[sampli,0]:.2f}/{losses[sampli,1]:.2f}')

# Exercise 4: Visualize the results

In [ ]:
# plot the losses
_,axs = plt.subplots(1,2,figsize=(10,4))


plt.tight_layout()
plt.show()

In [ ]:
# random starting tokens
randstarts = torch.randint(tokenizer.vocab_size,(numreps,1)).to(device)


# FREEZE: generate and store tokens



# TRAIN: same as above :)


# calculate the percentage
tokenUsage[1,0] = np.mean(100*np.isin(genTokensFreeze,top100))
tokenUsage[1,1] = np.mean(100*np.isin(genTokensTrain ,top100))

In [ ]:
print(tokenizer.decode(genTokensTrain))

In [ ]:
# visualize the results!

_,axs = plt.subplots(1,2,figsize=(10,3))

# show the pre-train token usage
axs[0].bar([.8,1.8],,width=.4,label='BEFORE')
axs[0].bar([1.2,2.2],,width=.4,label='AFTER')
axs[0].set(ylim=[min(tokenUsage.flatten())-2,max(tokenUsage.flatten())+2],xticks=[1,2],xlim=[.3,2.6],
           xticklabels=['FREEZE model','TRAIN model'],ylabel='Percent generated (%)',title='Common Moby Dick tokens generated')
axs[0].legend()

axs[1].bar([1,2],
axs[1].set(xticks=[1,2],xlim=[.3,2.6],xticklabels=['FREEZE model','TRAIN model'],
           ylabel='Change in generated tokens (%)',title='Post- minus pre-training')


plt.tight_layout()
plt.show()

In [ ]:
# how did the embeddings weights change?
plt.figure(figsize=(8,3))

plt.plot(delta_norm_em[:,0],linewidth=2,label='FREEZE')
plt.plot(delta_norm_em[:,1],linewidth=2,label='TRAIN')

plt.legend()
plt.gca().set(xlim=[0,num_samples],xlabel='Training sample',ylabel='Matrix difference norm')
plt.show()

In [ ]:
# Computation time
plt.bar
plt.gca().set(xticks=[1,2],xticklabels=['FREEZE','TRAIN'],ylabel='Computation time (s)',
              ylim=[min(timeFreeze,timeTrain)*.8,max(timeFreeze,timeTrain)*1.2],
              title=f'Computation time across {num_samples} training samples')
plt.show()